# 03 Q-agent hiperparametre analizi

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!ls '/content/drive/MyDrive/CASGEM-Egitim/Egitim-Part2/Day15-ReinforcementLearning/notebooks/01_taxi'

In [ ]:
import os
os.chdir('/content/drive/MyDrive/CASGEM-Egitim/Egitim-Part2/Day15-ReinforcementLearning/notebooks/01_taxi')

#### 👉RL ajanları hiper parametrelere son derece duyarlıdır.

#### 👉Hiper parametrelerin öğrenmeyi nasıl etkilediklerine dair daha iyi bir sezgi kazanmak için onlarla sistematik bir şekilde oynayalım.

**Alfa (learning rate) ve gama (discount factor) için farklı değerler kullanarak q-agent'ı eğitelim. Epsilon'a gelince, %10'da tutuyoruz.**

**Kodu temiz tutmak için q-agent tanımını src/q_agent.py içine ve eğitim döngüsünü
src/loops.py içindeki train() işlevinin içine yerleştirebiliriz.**


In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline
%config InlineBackend.figure_format = 'svg'

## Çevre (Environment) 🌎

In [ ]:
import gym
env = gym.make("Taxi-v3").env

## Q-agent 🤖🧠

In [ ]:
# No need to copy paste the same QAgent
# definition in every notebook, don't you think?
from src.q_agent import QAgent

# hyper-parameters
# RL problems are full of these hyper-parameters.
# For the moment, trust me when I set these values.
# We will later play with these and see how they impact learning.
alphas = [0.01, 0.1, 1]
gammas = [0.1, 0.6, 0.9]

## Training loop 🎡

In [ ]:
import pandas as pd

from src.loops import train

# exploration vs exploitation prob
# let's start with a constant probability of 10%.
epsilon = 0.1
n_episodes = 1000

results = pd.DataFrame()
for alpha in alphas:
    for gamma in gammas:
        
        print(f'alpha: {alpha}, gamma: {gamma}')
        agent = QAgent(env, alpha, gamma)
        
        _, timesteps, penalties = train(agent,
                                        env,
                                        n_episodes,
                                        epsilon)
        
        # collect timesteps and penalties for this pair
        # of hyper-parameters (alpha, gamma)
        results_ = pd.DataFrame()
        results_['timesteps'] = timesteps
        results_['penalties'] = penalties
        results_['alpha'] = alpha
        results_['gamma'] = gamma
        results = pd.concat([results, results_])

# index -> episode
results = results.reset_index().rename(
    columns={'index': 'episode'})

# add column with the 2 hyper-parameters
results['hyperparameters'] = [
    f'alpha={a}, gamma={g}'
    for (a, g) in zip(results['alpha'], results['gamma'])
]

**Her bir hiper parametre kombinasyonu için episode başına zaman adımlarını çizelim.**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig = plt.gcf()
fig.set_size_inches(12, 8)
sns.lineplot('episode', 'timesteps',
             hue='hyperparameters', data=results)

Grafik iddialı görünüyor ama biraz fazla gürültülü.

**Yine de gözlemleyebileceğimiz şey, alfa = 0.01 olduğunda öğrenmenin daha yavaş olduğudur. alfa (öğrenme hızı), her yinelemede q değerlerini ne kadar güncellediğimizi kontrol eder. Değerin çok küçük olması daha yavaş öğrenme anlamına gelir.**

Alfa = 0.01'i atayalım ve her hiper parametre kombinasyonu için 10 antrenman yapalım. Bu 10 çalıştırmayı kullanarak, 1'den 1000'e kadar her bölüm numarası (episode number) için zaman adımlarının ortalamasını alıyoruz.

Kodu daha temiz tutmak için src/loops.py dosyasında train_many_runs() işlevini oluşturalım:


In [ ]:
from src.loops import train_many_runs

alphas = [0.1, 1]
gammas = [0.1, 0.6, 0.9]

epsilon = 0.1
n_episodes = 1000
n_runs = 10

results = pd.DataFrame()
for alpha in alphas:
    for gamma in gammas:
        
        print(f'alpha: {alpha}, gamma: {gamma}')
        agent = QAgent(env, alpha, gamma)
        
        timesteps, penalties = train_many_runs(agent,
                                               env,
                                               n_episodes,
                                               epsilon,
                                               n_runs)
        
        # collect timesteps and penalties for this pair of
        # hyper-parameters (alpha, gamma)
        results_ = pd.DataFrame()
        results_['timesteps'] = timesteps
        results_['penalties'] = penalties
        results_['alpha'] = alpha
        results_['gamma'] = gamma
        results = pd.concat([results, results_])

# index -> episode
results = results.reset_index().rename(
    columns={'index': 'episode'})

results['hyperparameters'] = [
    f'alpha={a}, gamma={g}'
    for (a, g) in zip(results['alpha'], results['gamma'])]

In [ ]:
fig = plt.gcf()
fig.set_size_inches(12, 8)
sns.lineplot('episode', 'timesteps', hue='hyperparameters', data=results)

**Görünüşe göre alfa = 1.0 en iyi çalışan değer, gama ise daha az etkiye sahip gibi görünüyor.**

Hiper parametreleri ayarlamak zaman alıcı ve sıkıcı olabilir. Optuna gibi, az önceki manuel süreci otomatikleştirmek için mükemmel kütüphaneler de var. Şimdilik, yeni bulduğumuz değerlerle çalışalım.

epsilon = %10'da ne oluyor?
Mevcut %10'luk değer en iyisi mi?

Bunu kontrol edelim.

Bulunan en iyi alfa ve gama değerlerini alıyoruz.

**alfa = 1.0**
**gama = 0.9 (0,1 veya 0,6 da alabilirdik)**

**Ve farklı epsilonlarla antrenman yapalım = [0.01, 0.1, 0.9]**

In [ ]:
# best hyper-parameters so far
alpha = 1.0
gamma = 0.9

epsilons = [0.01, 0.10, 0.9]
n_runs = 10
n_episodes = 200

results = pd.DataFrame()
for epsilon in epsilons:
        
    print(f'epsilon: {epsilon}')
    agent = QAgent(env, alpha, gamma)

    timesteps, penalties = train_many_runs(agent,
                                           env,
                                           n_episodes,
                                           epsilon,
                                           n_runs)

    # collect timesteps and penalties for this pair of
    # hyper-parameters (alpha, gamma)
    results_ = pd.DataFrame()
    results_['timesteps'] = timesteps
    results_['penalties'] = penalties
    results_['epsilon'] = epsilon
    results = pd.concat([results, results_])

# index -> episode
results = results.reset_index().rename(columns={'index': 'episode'})

## Ve ortaya çıkan zaman adımlarını ve ceza eğrilerini çizelim:

In [ ]:
fig = plt.gcf()
fig.set_size_inches(12, 8)
sns.lineplot('episode', 'timesteps', hue='epsilon', data=results)
plt.show()

fig = plt.gcf()
fig.set_size_inches(12, 8)
sns.lineplot('episode', 'penalties', hue='epsilon', data=results)

Gördüğünüz gibi, hem epsilon = 0.01 hem de epsilon = 0.1, keşif ve sömürü arasında doğru dengeyi tuttukları için eşit derecede iyi çalışıyor gibi görünüyor.

Öte yandan, epsilon = 0.9 çok büyük bir değerdir, eğitim sırasında "çok fazla" rastgeleliğe neden olur ve q-matriksimizin optimal olana yakınsamasını engeller. 

Performansın bölüm başına yaklaşık 250 zaman adımında nasıl plato yaptığını gözlemleyin.

Genel olarak, epsilon hiper parametresini seçmek için en iyi strateji ilerleyici epsilon bozunumudur (epsilon-decay). Yani, eğitimin başlangıcında, ajan q-değeri tahmininden çok emin olmadığında, mümkün olduğu kadar çok durumun ziyaret edilmesi en iyisidir ve bunun için büyük bir epsilon harika bir seçimdir (örneğin %50).

Eğitim ilerledikçe ve aracı q-değeri tahminini iyileştirdikçe, o kadar fazla araştırma yapmak artık optimal değildir. Bunun yerine, epsilon'u azaltarak, ajan q-değerlerini mükemmelleştirmeyi ve ince ayar yapmayı öğrenebilir, böylece onları optimal değerlere daha hızlı yakınsayabilir. 

Epsilon'un çok büyük olması, epsilon = 0.9 için gördüğümüz gibi yakınsama sorunlarına neden olabilir.
